## Exploring Rate Remapping
WH late July 2019

In [1]:
import ratterdam_CoreDataStructures as Core
import ratterdam_ParseBehavior as Parse
import numpy as np
from scipy.stats import sem
import utility_fx as util
import os
from matplotlib.backends.backend_pdf import PdfPages
import matplotlib.gridspec as gridspec
from matplotlib import pyplot as plt
import ratterdam_Defaults as Def
import ratterdam_visBasic as Vis
from scipy.interpolate import interp1d

In [2]:
%qtconsole --style native
%matplotlib qt5

In [32]:
datafile = "E:\\Ratterdam\\R808\\R808_Beltway_D7\\"
expCode = "BRD7"

beltwayAlleys = [16,17,3,1,5,7,8,10,11]

ratplot = Vis.BasicRateMaps()

alleyTracking, alleyVisits,  txtVisits, p_sess, ts_sess = Parse.getDaysBehavioralData(datafile, expCode)
stimfiles = Parse.getStimFileName(datafile)
stimData = Parse.loadBeltwayData(datafile,stimfiles,expCode)


#alleySwapTS = Parse.beltway_getLapTimes(datafile, stimData, expCode)

cmap = util.makeCustomColormap()

In [43]:
unit = Core.UnitData("TT14\\cl-maze1.3", datafile, expCode, Def.alleyBounds, alleyVisits, txtVisits, p_sess, ts_sess)
unit.loadData_raw()

C:\Users\whockei1\Google Drive\KnierimLab\Ratterdam\Code\ratterdam_CoreDataStructures.py:170: RuntimeWarning: divide by zero encountered in reciprocal
  n = (hs*np.reciprocal(ho))*30
C:\Users\whockei1\Google Drive\KnierimLab\Ratterdam\Code\ratterdam_CoreDataStructures.py:170: RuntimeWarning: invalid value encountered in multiply
  n = (hs*np.reciprocal(ho))*30
C:\Users\whockei1\Google Drive\KnierimLab\Ratterdam\Code\ratterdam_CoreDataStructures.py:176: RuntimeWarning: divide by zero encountered in reciprocal
  n = (ls* np.reciprocal(lo)) * 30
C:\Users\whockei1\Google Drive\KnierimLab\Ratterdam\Code\ratterdam_CoreDataStructures.py:176: RuntimeWarning: invalid value encountered in multiply
  n = (ls* np.reciprocal(lo)) * 30
C:\Users\whockei1\Google Drive\KnierimLab\Ratterdam\Code\utility_fx.py:324: RuntimeWarning: invalid value encountered in true_divide
  Z=VV/WW


In [23]:
firstSpikeTimes = {'A':[], 'B':[], 'C':[]}
for alley in beltwayAlleys:
    for i,visit in enumerate(unit.alleys[alley]):
        if visit['spikes'].shape[0] > 0:
            firstspk = visit['spikes'][0,0]
            visittime = alleyVisits[alley-1][i][0]
            diff = (firstspk - visittime)/1e6
            firstSpikeTimes[visit['metadata']['stimulus']].append(diff)
data = []          
for i,(t,c) in enumerate(zip(['A','B','C'],['r','b','g'])):
    data.append(firstSpikeTimes[t])
    print(np.median(firstSpikeTimes[t]))
    print(sem(firstSpikeTimes[t]))
plt.boxplot(data)

826.9448975
98.2654682664
683.922568
83.8098647577
641.407854
91.545200979


{'boxes': [<matplotlib.lines.Line2D at 0x2570e333da0>,
 'caps': [<matplotlib.lines.Line2D at 0x2570e33d908>,
 'fliers': [<matplotlib.lines.Line2D at 0x2570e344668>,
 'means': [],
 'medians': [<matplotlib.lines.Line2D at 0x2570e344208>,
 'whiskers': [<matplotlib.lines.Line2D at 0x2570e333f28>,
  <matplotlib.lines.Line2D at 0x2570e35d320>]}

-------------------------------------------------------------------------

In [12]:
spikes = {'A':[], 'B':[], 'C':[]}
for alley in [7]:
    for i,visit in enumerate(unit.alleys[alley]):
        if visit['spikes'].shape[0] > 0:
            diffs = (np.diff(visit['spikes'][:,0])/1e3)/visit['spikes'].shape[0]
            spikes[visit['metadata']['stimulus']] = np.hstack((spikes[visit['metadata']['stimulus']], diffs))

In [13]:
for i,(t,c) in enumerate(zip(['A','B','C'],['r','b','g'])):
    plt.scatter([i]*spikes[t].shape[0], spikes[t],c=c)

In [14]:
#fig, ax = plt.subplots(1,3)
for i,(t,c) in enumerate(zip(['A','B','C'],['r','b','g'])):
    plt.hist(spikes[t],bins=50,color=c,alpha=0.3)

In [15]:
data = []
for i,(t,c) in enumerate(zip(['A','B','C'],['r','b','g'])):
    data.append(spikes[t])
plt.boxplot(data)

{'boxes': [<matplotlib.lines.Line2D at 0x1f2a3fbfa90>,
 'caps': [<matplotlib.lines.Line2D at 0x1f2a3fcb5f8>,
 'fliers': [<matplotlib.lines.Line2D at 0x1f2a4e12358>,
 'means': [],
 'medians': [<matplotlib.lines.Line2D at 0x1f2a3fcbeb8>,
 'whiskers': [<matplotlib.lines.Line2D at 0x1f2a3fbfc18>,
  <matplotlib.lines.Line2D at 0x1f2a4e21f98>]}

In [17]:
alley = 7
for v in unit.alleys[alley]:
    plt.scatter(v['occs'][:,1], v['occs'][:,2])
for i in range(1,18):
    ratplot.drawAlleyBounds(plt.gca(),i)

In [6]:
plt.scatter(alleyTracking[alley-1][:,1], alleyTracking[alley-1][:,2])
for i in range(1,18):
    ratplot.drawAlleyBounds(plt.gca(),i)

In [7]:
 v = unit.alleys[8][3]
plt.scatter(v['occs'][:,1], v['occs'][:,2])
for i in range(1,18):
    ratplot.drawAlleyBounds(plt.gca(),i)

In [21]:
alley = 7
ncols = 5
fig, ax = plt.subplots(int(np.ceil(len(unit.alleys[alley])/ncols)), ncols)
for i,visit in enumerate(unit.alleys[alley]):
    fig.axes[i].scatter(visit['spikes'][:,1], visit['spikes'][:,2])
    fig.axes[i].plot(visit['occs'][:,1], visit['occs'][:,2], alpha=0.7, color='k')
    fig.axes[i].set_title(visit['metadata']['stimulus'])
    ratplot.drawAlleyBounds(fig.axes[i], alley)

## Temporal Field Modulation

In [44]:
winSz = 5
alley = 5# whole track numbering
visits = {txt:np.empty((0, Def.singleAlleyBins[0]-1)) for txt in ['A','B','C']}
visitIdx = {txt:[] for txt in ['A','B','C']}

for i,visit in enumerate(unit.alleys[alley]):
    txt  = visit['metadata']['stimulus']
    visits[txt] = np.vstack((visits[txt], visit['ratemap1d']))
    visitIdx[txt].append(i)

In [11]:
def txtMod_testStat(x,y,statchoice):
    """
    Compute a given test statistic of rate modulation
    between avg linearized response to txt X, Y
    """
    if statchoice == 'meandiff':
        stat = np.nanmean(np.abs(x-y))
    elif statchoice == 'maxdiff':
        stat = np.nanmax(np.abs(x-y))
    elif statchoice == 'auc':
        stat = np.trapz(x-y)
    return stat

In [45]:
stattype = 'auc'
mx = max([len(visitIdx[t]) for t in ['A','B','C']])
numWin = (mx-winSz)+1
runningmaxes = []
for i in range(numWin):
    txtWin = {txt:np.empty((0, Def.singleAlleyBins[0]-1)) for txt in ['A','B','C']}
    for txt in ['A','B','C']:
        try:
            txtWin[txt] = visits[txt][0+i:winSz+i]
        except:
            txtWin[txt] = visits[txt][0+i:]
            
    avgs = {txt:np.empty((0, Def.singleAlleyBins[0]-1)) for txt in ['A','B','C']}
    for txt in ['A', 'B', 'C']:
        mask = np.ma.masked_invalid(txtWin[txt])
        avgs[txt] = mask.mean(axis=0) # ignores inf and nan
    ab = txtMod_testStat(avgs['A'], avgs['B'], statchoice=stattype)
    cb = txtMod_testStat(avgs['C'], avgs['B'], statchoice=stattype)
    ac = txtMod_testStat(avgs['A'], avgs['C'], statchoice=stattype)
    winmax = np.nanmax([ab, cb, ac])
    runningmaxes.append(winmax)

In [46]:
toshuff = {txt:np.empty((0, Def.singleAlleyBins[0]-1)) for txt in ['A','B','C']}
visitIdx = {txt:[] for txt in ['A','B','C']}

for i,visit in enumerate(unit.alleys[alley]):
    txt  = visit['metadata']['stimulus']
    toshuff[txt] = np.vstack((visits[txt], visit['ratemap1d']))
    visitIdx[txt].append(i)
    
shuffmaxes = []
for i in range(1000):
    for txt in ['A','B','C']:
        np.random.shuffle(toshuff[txt])
    runningmaxesshuff = []
    for i in range(numWin):
        txtWin = {txt:np.empty((0, Def.singleAlleyBins[0]-1)) for txt in ['A','B','C']}
        for txt in ['A','B','C']:
            try:
                txtWin[txt] = toshuff[txt][0+i:winSz+i]
            except:
                txtWin[txt] = toshuff[txt][0+i:]

        avgs = {txt:np.empty((0, Def.singleAlleyBins[0]-1)) for txt in ['A','B','C']}
        for txt in ['A', 'B', 'C']:
            mask = np.ma.masked_invalid(txtWin[txt])
            avgs[txt] = mask.mean(axis=0) # ignores inf and nan
        ab = txtMod_testStat(avgs['A'], avgs['B'], statchoice=stattype)
        cb = txtMod_testStat(avgs['C'], avgs['B'], statchoice=stattype)
        ac = txtMod_testStat(avgs['A'], avgs['C'], statchoice=stattype)
        winmax = np.nanmax([ab, cb, ac])
        runningmaxesshuff.append(winmax)
    shuffmaxes.append(runningmaxesshuff)
shuffmaxes=np.asarray(shuffmaxes)

In [42]:
plt.plot(np.nanpercentile(shuffmaxes,97.5,axis=0),'k')
plt.plot(np.nanpercentile(shuffmaxes,2.5,axis=0),'k')
plt.plot(runningmaxes)
plt.title(f"{expCode} {unit.name}")

Text(0.5,1,'BRD7 TT6_0001cl-maze1.3')